# Armaan Kapoor, ECE-464, PS1 (Sailors)

Get sailor, boat and reservation data from class repo: [Create a Sailors and Boats dataset in Python
@eugsokolov](https://github.com/eugsokolov/ece464-databases/blob/master/sailors/sailors.py)

In [1]:
sailors = [ # sid, sname, rating, age
    (22, "dusting", 7, 45.0),
    (23, "emilio", 7, 45.0),
    (24, "scruntus", 1, 33.0),
    (29, "brutus", 1, 33.0),
    (31, "lubber", 8, 55.5),
    (32, "andy", 8, 25.5),
    (35, "figaro", 8, 55.5),
    (58, "rusty", 10, 35),
    (59, "stum", 8, 25.5),
    (60, "jit", 10, 35),
    (61, "ossola", 7, 16),
    (62, "shaun", 10, 35),
    (64, "horatio", 7, 16),
    (71, "zorba", 10, 35),
    (74, "horatio", 9, 25.5),
    (85, "art", 3, 25.5),
    (88, "kevin", 3, 25.5),
    (89, "will", 3, 25.5),
    (90, "josh", 3, 25.5),
    (95, "bob", 3, 63.5),
]

boats = [ # bid, bname, color, length
    (101, "Interlake", "blue", 45),
    (102, "Interlake", "red", 45),
    (103, "Clipper", "green", 40),
    (104, "Clipper", "red", 40),
    (105, "Marine", "red", 35),
    (106, "Marine", "green", 35),
    (107, "Marine", "blue", 35),
    (108, "Driftwood", "red", 35),
    (109, "Driftwood", "blue", 35),
    (110, "Klapser", "red", 30),
    (111, "Sooney", "green", 28),
    (112, "Sooney", "red", 28),
]

reserves = [ # sid, bid, day
    (22, 101, "1998-10-10"),
    (22, 102, "1998-10-10"),
    (22, 103, "1998-08-10"),
    (22, 104, "1998-07-10"),
    (23, 104, "1998-10-10"),
    (23, 105, "1998-11-10"),
    (24, 104, "1998-10-10"),
    (31, 102, "1998-11-10"),
    (31, 103, "1998-11-06"),
    (31, 104, "1998-11-12"),
    (35, 104, "1998-08-10"),
    (35, 105, "1998-11-06"),
    (59, 105, "1998-07-10"),
    (59, 106, "1998-11-12"),
    (59, 109, "1998-11-10"),
    (60, 106, "1998-09-05"),
    (60, 106, "1998-09-08"),
    (60, 109, "1998-07-10"),
    (61, 112, "1998-09-08"),
    (62, 110, "1998-11-06"),
    (64, 101, "1998-09-05"),
    (64, 102, "1998-09-08"),
    (74, 103, "1998-09-08"),
    (88, 107, "1998-09-08"),
    (88, 110, "1998-09-05"),
    (88, 110, "1998-11-12"),
    (88, 111, "1998-09-08"),
    (89, 108, "1998-10-10"),
    (89, 109, "1998-08-10"),
    (90, 109, "1998-10-10"),
]


In [2]:
from __future__ import print_function

import datetime
import os

import pandas as pd
from sqlalchemy import (
    Column,
    DateTime,
    ForeignKey,
    Integer,
    String,
    and_,
    create_engine,
    func,
    or_,
    select,
)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import backref, relationship, sessionmaker

# Define the location of the SQLite database
location = "sailors.sqlite"

# Check if the database file exists and delete it if it does
if os.path.exists(location):
    os.remove(location)

# Create a new engine instance
engine = create_engine(f"sqlite:///{location}")

Base = declarative_base()


class Sailor(Base):
    __tablename__ = "sailors"

    sid = Column(Integer, primary_key=True)
    sname = Column(String)
    rating = Column(Integer)
    age = Column(Integer)

    def __repr__(self):
        return "<Sailor(id=%s, name='%s', rating=%s, age=%s)>" % (
            self.sid,
            self.sname,
            self.rating,
            self.age,
        )


class Boat(Base):
    __tablename__ = "boats"

    bid = Column(Integer, primary_key=True)
    bname = Column(String)
    color = Column(String)
    length = Column(Integer)

    reservations = relationship(
        "Reservation", backref=backref("boat", cascade="delete")
    )

    def __repr__(self):
        return "<Boat(id=%s, name='%s', color=%s, length=%s)>" % (
            self.bid,
            self.bname,
            self.color,
            self.length,
        )


class Reservation(Base):
    __tablename__ = "reserves"
    sid = Column(Integer, ForeignKey("sailors.sid"), primary_key=True)
    bid = Column(Integer, ForeignKey("boats.bid"), primary_key=True)
    day = Column(DateTime, primary_key=True)

    sailor = relationship("Sailor", backref="reservations")

    def __repr__(self):
        return "<Reservation(sailor_id=%s, boat_id=%s, day=%s)>" % (
            self.sid,
            self.bid,
            self.day,
        )


Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

# Inserting from the data in the cell above
sailor_data = [
    Sailor(sid=sid, sname=sname, rating=rating, age=age)
    for sid, sname, rating, age in sailors
]

boat_data = [
    Boat(bid=bid, bname=bname, color=color, length=length)
    for bid, bname, color, length in boats
]

reservation_data = [
    Reservation(sid=sid, bid=bid, day=datetime.datetime.strptime(day, "%Y-%m-%d"))
    for sid, bid, day in reserves
]

session.add_all(sailor_data + boat_data + reservation_data)
session.commit()

## Part 1
---

### Problem 1.
List, for every boat, the number of times it has been reserved, excluding those boats that have never been reserved (list the id and the name).

In [3]:
q1 = """
SELECT boats.bid, boats.bname, COUNT(reserves.bid) AS num_reservations
FROM boats
JOIN reserves ON boats.bid = reserves.bid
GROUP BY boats.bid, boats.bname
HAVING COUNT(reserves.bid) > 0
"""

q1_res = list(session.execute(q1)) # store for future ORM test case comparison
pd.read_sql(q1, session.bind)


,bid,bname,num_reservations
0,101,Interlake,2
1,102,Interlake,3
2,103,Clipper,3
3,104,Clipper,5
4,105,Marine,3
5,106,Marine,3
6,107,Marine,1
7,108,Driftwood,1
8,109,Driftwood,4
9,110,Klapser,3


### Problem 2.
List those sailors who have reserved every red boat (list the id and the name).

In [4]:
# I am interpreting this as "list sailors that have reserved all red boats"
q2a = """
SELECT sailors.sid, sailors.sname
FROM sailors
JOIN reserves ON sailors.sid = reserves.sid
JOIN boats ON reserves.bid = boats.bid AND boats.color = 'red'
GROUP BY sailors.sid, sailors.sname
HAVING COUNT(DISTINCT boats.bid) = (SELECT COUNT(*) FROM boats WHERE color = 'red')
"""

q2a_res = list(session.execute(q2a))
pd.read_sql_query(q2a, session.bind)

,sid,sname


There are none!

In [5]:
# Alternative interpretation: to find all the sailor who have reserved at least one red boat.
q2b = """
SELECT sailors.sid, sailors.sname
FROM sailors
JOIN reserves ON sailors.sid = reserves.sid
JOIN boats ON reserves.bid = boats.bid AND boats.color = 'red'
GROUP BY sailors.sid, sailors.sname
"""
q2b_res = list(session.execute(q2b))
pd.read_sql_query(q2b, session.bind)

,sid,sname
0,22,dusting
1,23,emilio
2,24,scruntus
3,31,lubber
4,35,figaro
5,59,stum
6,61,ossola
7,62,shaun
8,64,horatio
9,88,kevin


### Problem 3.
List those sailors who have reserved only red boats.

In [6]:
q3 = """
SELECT DISTINCT sailors.sid, sailors.sname
FROM sailors
JOIN reserves ON sailors.sid = reserves.sid
WHERE sailors.sid NOT IN (
    SELECT DISTINCT reserves.sid
    FROM reserves
    JOIN boats ON reserves.bid = boats.bid
    WHERE boats.color <> 'red'
)
"""

q3_res = list(session.execute(q3))
pd.read_sql_query(q3, session.bind)

,sid,sname
0,23,emilio
1,24,scruntus
2,35,figaro
3,61,ossola
4,62,shaun


### Problem 4.
For which boat are there the most reservations?

In [7]:
q4 = """
SELECT boats.bid, boats.bname, COUNT(reserves.bid) AS num_reservations
FROM boats
JOIN reserves ON boats.bid = reserves.bid
GROUP BY boats.bid, boats.bname
HAVING COUNT(reserves.bid) = (
    SELECT MAX(reservation_counts.num_reservations)
    FROM (
        SELECT COUNT(reserves.bid) AS num_reservations
        FROM reserves
        GROUP BY reserves.bid
    ) AS reservation_counts
)
"""

q4_res = list(session.execute(q4))
pd.read_sql_query(q4, session.bind)

,bid,bname,num_reservations
0,104,Clipper,5


### Problem 5.
Select all sailors who have never reserved a red boat.

In [8]:
q5 = """
SELECT sailors.sid, sailors.sname
FROM sailors
WHERE NOT EXISTS (
    SELECT 1
    FROM reserves
    JOIN boats ON reserves.bid = boats.bid
    WHERE boats.color = 'red'
    AND reserves.sid = sailors.sid
)
"""

q5_res = list(session.execute(q5))
pd.read_sql_query(q5, session.bind)

,sid,sname
0,29,brutus
1,32,andy
2,58,rusty
3,60,jit
4,71,zorba
5,74,horatio
6,85,art
7,90,josh
8,95,bob


### Problem 6.
Find the average age of sailors with a rating of 10.

In [9]:
q6 = """
SELECT AVG(age) AS average_age
FROM sailors
WHERE rating = 10
"""

q6_res = list(session.execute(q6))
pd.read_sql_query(q6, session.bind)

,average_age
0,35.0


### Problem 7.
For each rating, find the name and id of the youngest sailor.

In [10]:
q7 = """
SELECT s.sid, s.sname, s.rating, s.age
FROM sailors s
INNER JOIN (
    SELECT rating, MIN(age) AS min_age
    FROM sailors
    GROUP BY rating
) AS min_ages ON s.rating = min_ages.rating AND s.age = min_ages.min_age
ORDER BY s.rating
"""

q7_res = list(session.execute(q7))
pd.read_sql_query(q7, session.bind)

,sid,sname,rating,age
0,24,scruntus,1,33.0
1,29,brutus,1,33.0
2,85,art,3,25.5
3,88,kevin,3,25.5
4,89,will,3,25.5
5,90,josh,3,25.5
6,61,ossola,7,16.0
7,64,horatio,7,16.0
8,32,andy,8,25.5
9,59,stum,8,25.5


### Problem 8.
Select, for each boat, the sailor who made the highest number of reservations for that boat.

In [11]:
q8 = """
SELECT r1.bid, r1.sid, r1.num_reservations
FROM (
    SELECT reserves.bid, reserves.sid, COUNT(*) AS num_reservations
    FROM reserves
    GROUP BY reserves.bid, reserves.sid
) r1
LEFT JOIN (
    SELECT reserves.bid, reserves.sid, COUNT(*) AS num_reservations
    FROM reserves
    GROUP BY reserves.bid, reserves.sid
) r2 ON r1.bid = r2.bid AND r1.num_reservations < r2.num_reservations
WHERE r2.bid IS NULL
"""

q8_res = list(session.execute(q8))
pd.read_sql_query(q8, session.bind)

,bid,sid,num_reservations
0,101,22,1
1,102,22,1
2,103,22,1
3,104,22,1
4,104,23,1
5,105,23,1
6,104,24,1
7,102,31,1
8,103,31,1
9,104,31,1


## Part 2:
Represent the sailors and boats schema using an ORM - I prefer SQLAlchemy but students have the freedom to choose their own language and ORM. Show that it is fully functional by writing tests with a testing framework using the data from part 1

In [12]:
def num_reservations_per_boat(session):
    return (
        session.query(
            Boat.bid, Boat.bname, func.count(Reservation.bid).label("num_reservations")
        )
        .join(Reservation, Boat.bid == Reservation.bid)
        .group_by(Boat.bid)
        .having(func.count(Reservation.bid) > 0)
        .all()
    )


def sailors_reserved_all_red_boats(session):
    red_boats_count = (
        session.query(func.count(Boat.bid)).filter(Boat.color == "red").scalar()
    )
    subq = (
        session.query(Reservation.sid)
        .join(Boat)
        .filter(Boat.color == "red")
        .group_by(Reservation.sid)
        .having(func.count(Boat.bid) == red_boats_count)
        .subquery()
    )

    result = session.query(Sailor.sid, Sailor.sname).join(
        subq, Sailor.sid == subq.c.sid
    )

    return result.all()


def sailors_reserved_at_least_one_red_boat(session):
    return (
        session.query(Sailor.sid, Sailor.sname)
        .join(Reservation)
        .join(Boat)
        .filter(Boat.color == "red")
        .group_by(Sailor.sid)
        .all()
    )


def sailors_reserved_only_red_boats(session):
    non_red_boats_sailors_subq = (
        session.query(Reservation.sid)
        .join(Boat)
        .filter(Boat.color != "red")
        .distinct()
        .subquery("non_red_sailors")
    )

    result = (
        session.query(Sailor.sid, Sailor.sname)
        .distinct()
        .join(Reservation)
        .filter(~Sailor.sid.in_(non_red_boats_sailors_subq))
    )

    return result.all()


def most_reserved_boats(session):
    # Count reservations per boat
    reservation_counts = (
        session.query(
            Reservation.bid.label("bid"),
            func.count(Reservation.bid).label("num_reservations"),
        )
        .group_by(Reservation.bid)
        .subquery("reservation_counts")
    )

    # Find the maximum number of reservations
    max_reservations = session.query(
        func.max(reservation_counts.c.num_reservations).label("max_reservations")
    ).scalar()

    result = (
        session.query(
            Boat.bid, Boat.bname, func.count(Reservation.bid).label("num_reservations")
        )
        .join(Reservation, Boat.bid == Reservation.bid)
        .group_by(Boat.bid, Boat.bname)
        .having(func.count(Reservation.bid) == max_reservations)
        .all()
    )

    return result


def sailors_never_reserved_red_boat(session):
    # Find sailors who have reserved a red boat
    reserved_red_boat_subq = (
        session.query(Reservation.sid).join(Boat).filter(Boat.color == "red").subquery()
    )

    # Find sailors who have never reserved a red boat
    result = session.query(Sailor.sid, Sailor.sname).filter(
        ~Sailor.sid.in_(reserved_red_boat_subq)
    )

    return result.all()


def average_age_rating_10(session):
    # Calculate the average age of sailors with a rating of 10
    average_age = (
        session.query(func.avg(Sailor.age).label("average_age"))
        .filter(Sailor.rating == 10)
        .scalar()
    )

    return average_age


def youngest_sailors_by_rating(session):
    # Subquery to find the minimum age for each rating
    min_ages_subq = (
        session.query(
            Sailor.rating.label("rating"), func.min(Sailor.age).label("min_age")
        )
        .group_by(Sailor.rating)
        .subquery("min_ages")
    )

    # Main query to find the youngest sailors for each rating
    youngest_sailors = (
        session.query(Sailor.sid, Sailor.sname, Sailor.rating, Sailor.age)
        .join(
            min_ages_subq,
            (Sailor.rating == min_ages_subq.c.rating)
            & (Sailor.age == min_ages_subq.c.min_age),
        )
        .order_by(Sailor.rating)
    )

    return youngest_sailors.all()


def top_reservist_per_boat(session):
    # Count the number of reservations per sailor and boat
    reservation_counts = (
        session.query(
            Reservation.bid.label("bid"),
            Reservation.sid.label("sid"),
            func.count("*").label("num_reservations"),
        )
        .group_by(Reservation.bid, Reservation.sid)
        .subquery("reservation_counts")
    )

    # Find the maximum number of reservations for each boat
    max_reservations_per_boat = (
        session.query(
            reservation_counts.c.bid.label("bid"),
            func.max(reservation_counts.c.num_reservations).label("max_reservations"),
        )
        .group_by(reservation_counts.c.bid)
        .subquery("max_reservations_per_boat")
    )

    # Join the reservation counts with the max reservations per boat to find the sailor with most res for each boat
    top_reservists = (
        session.query(
            reservation_counts.c.bid,
            reservation_counts.c.sid,
            reservation_counts.c.num_reservations,
        )
        .join(
            max_reservations_per_boat,
            (reservation_counts.c.bid == max_reservations_per_boat.c.bid)
            & (
                reservation_counts.c.num_reservations
                == max_reservations_per_boat.c.max_reservations
            ),
        )
        .order_by(reservation_counts.c.bid)
    )

    return top_reservists.all()


try:
    assert num_reservations_per_boat(session) == q1_res
    assert sailors_reserved_all_red_boats(session) == q2a_res
    assert sailors_reserved_at_least_one_red_boat(session) == q2b_res
    assert sailors_reserved_only_red_boats(session) == q3_res
    assert most_reserved_boats(session) == q4_res
    assert sailors_never_reserved_red_boat(session) == q5_res
    assert average_age_rating_10(session) == q6_res[0][0]
    assert youngest_sailors_by_rating(session) == q7_res
    assert set(top_reservist_per_boat(session)) == set(q8_res)  # order is not important
    print("ALL QUERIES PASSED!")
except AssertionError:
    print("SUMN WENT WRONG")

ALL QUERIES PASSED!


/var/folders/nx/n1nd7k552014npzf4p5zpxh00000gn/T/ipykernel_38925/3300327185.py:57: SAWarning: Coercing Subquery object into a select() for use in IN(); please pass a select() construct explicitly
  .filter(~Sailor.sid.in_(non_red_boats_sailors_subq))
/var/folders/nx/n1nd7k552014npzf4p5zpxh00000gn/T/ipykernel_38925/3300327185.py:100: SAWarning: Coercing Subquery object into a select() for use in IN(); please pass a select() construct explicitly
  ~Sailor.sid.in_(reserved_red_boat_subq)


## Part 3
Tax season is around the corner and Boats4Pirates LLC is in deep shit with the IRS.

1. Let's add ```deposit ($)``` as a new attribute of our reservation records.
2. Add ```market value ($)``` and ```condition (1-10)``` to boat table.
3. Add ```money spent ($)``` to the sailor table.

Boats4Pirates now keeps track of how much they charged sailors to reserve a boat for the day. Using OpenCV, they successfully ported their pen and paper records to a digital database.

Boats4Pirates hired a surveyour to estimate mkt value for each boat in their fleet.

By keeping track of the ammount of money spent by sailors, B4P will reward their frequent spenders with bonus rewards and perks.

In [13]:
import datetime
import random

from sqlalchemy import Column, DateTime, Float, Integer, String, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

random.seed(0)  # set random seed for reproducibility


class Reservation(Base):
    __tablename__ = "reserves"
    sid = Column(Integer, primary_key=True)
    bid = Column(Integer, primary_key=True)
    day = Column(DateTime, primary_key=True)
    price = Column(Float)  # New attribute for price ($)


class Boat(Base):
    __tablename__ = "boats"
    bid = Column(Integer, primary_key=True)
    bname = Column(String)
    color = Column(String)
    length = Column(Integer)
    market_value = Column(Float)  # New attribute for market value ($)
    condition = Column(Integer)  # New attribute for condition (1-10)


class Sailor(Base):
    __tablename__ = "sailors"
    sid = Column(Integer, primary_key=True)
    sname = Column(String)
    rating = Column(Integer)
    age = Column(Integer)
    money_spent = Column(Float)  # New attribute for money spent ($)


# Generate sample data
def generate_sample_data():
    global sailors, boats, reserves
    # Update Sailor data with money spent
    sailors_with_money_spent = []
    for sailor in sailors:
        money_spent = round(
            random.uniform(100, 1000), 2
        )  # Generating random money spent
        sailors_with_money_spent.append((*sailor, money_spent))

    # Update Boat data with market value and condition
    boats_with_value_and_condition = []
    for boat in boats:
        market_value = round(
            random.uniform(5000, 20000), 2
        )  # Generating random market value
        condition = random.randint(1, 10)  # Generating random condition
        boats_with_value_and_condition.append((*boat, market_value, condition))

    # Update Reservation data with price
    reserves_with_price = []
    for reserve in reserves:
        price = round(random.uniform(30, 200), 2)  # Generating random price
        reserves_with_price.append((*reserve, price))

    return sailors_with_money_spent, boats_with_value_and_condition, reserves_with_price


engine = create_engine("sqlite:///:memory:")

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

sailors_with_money_spent, boats_with_value_and_condition, reserves_with_price = (
    generate_sample_data()
)

sailor_data = [
    Sailor(sid=sid, sname=sname, rating=rating, age=age, money_spent=money_spent)
    for sid, sname, rating, age, money_spent in sailors_with_money_spent
]
boat_data = [
    Boat(
        bid=bid,
        bname=bname,
        color=color,
        length=length,
        market_value=market_value,
        condition=condition,
    )
    for bid, bname, color, length, market_value, condition in boats_with_value_and_condition
]
reservation_data = [
    Reservation(
        sid=sid, bid=bid, day=datetime.datetime.strptime(day, "%Y-%m-%d"), price=price
    )
    for sid, bid, day, price in reserves_with_price
]

session.add_all(sailor_data + boat_data + reservation_data)
session.commit()

session.close()

In [14]:
# print reservation data as a pandas dataframe
pd.read_sql_query("SELECT * FROM reserves", engine)

,sid,bid,day,price
0,22,101,1998-10-10 00:00:00.000000,71.46
1,22,102,1998-10-10 00:00:00.000000,85.28
2,22,103,1998-08-10 00:00:00.000000,177.98
3,22,104,1998-07-10 00:00:00.000000,62.48
4,23,104,1998-10-10 00:00:00.000000,126.48
5,23,105,1998-11-10 00:00:00.000000,70.56
6,24,104,1998-10-10 00:00:00.000000,194.48
7,31,102,1998-11-10 00:00:00.000000,166.54
8,31,103,1998-11-06 00:00:00.000000,106.15
9,31,104,1998-11-12 00:00:00.000000,43.68


Currently the randomly populated data is counterintuitive and logically inaccurate. 

1. ```Sailor.money_spent``` should be the sum of all the ```Reservation.price``` enteries matched by ```Reservation.sid```.

3. ```Reservation.price``` should be a function of ```Boat.market_value```, and ```Sailor.rating```, ```Sailor.age```, and ```Sailor.money_spent```

4. ```Boat.market_value``` should be a function of ```Boat.length``` and perhaps ```Boat.condition```


In [15]:
# Im moving to pandas to do some fire ETL.
sailors_df = pd.DataFrame(sailors_with_money_spent, columns=["sid", "sname", "rating", "age", "money_spent"])
boats_df = pd.DataFrame(boats_with_value_and_condition, columns=["bid", "bname", "color", "length", "market_value", "condition"])
reserves_df = pd.DataFrame(reserves_with_price, columns=["sid", "bid", "day", "down_payment"])

In [16]:
# Updating Boat.market_value to make it a function of length and condition
boats_df["market_value"] = (boats_df["length"] ** 2) * boats_df["condition"] * 10
boats_df

,bid,bname,color,length,market_value,condition
0,101,Interlake,blue,45,40500,2
1,102,Interlake,red,45,121500,6
2,103,Clipper,green,40,32000,2
3,104,Clipper,red,40,96000,6
4,105,Marine,red,35,49000,4
5,106,Marine,green,35,98000,8
6,107,Marine,blue,35,110250,9
7,108,Driftwood,red,35,110250,9
8,109,Driftwood,blue,35,24500,2
9,110,Klapser,red,30,63000,7


We want to ideally to update reservation down_payment to be a function of boat price, sailor age, sailor rating, and the ammount of money spent by that sailor.

Circularly, we need to to update Sailor.money_spent to be logically consistent with the reservation data. For each sailor, the sum of the price of all reservations made by that sailor.

This is tricky...

We can proceed by ignoring sailor.money_spent and factoring it in later.

In [17]:
reserves_df = reserves_df.merge(boats_df[["bid", "market_value"]], on="bid")
reserves_df = reserves_df.merge(sailors_df[["sid", "age", "rating"]], on="sid")
# down_payment is market_value * (rating / 10) * (age / 100)
reserves_df["down_payment"] = (
    reserves_df["market_value"]
    * (reserves_df["rating"] / 10)
    * (reserves_df["age"] / 100)
)  # After age = 100, additional age will actually start to increase down_payment.
# This is funny but i will leave it as is.

# remove market_value column
reserves_df = reserves_df.drop(columns=["market_value"])
reserves_df

,sid,bid,day,down_payment,age,rating
0,22,101,1998-10-10,12757.500,45.0,7
1,22,102,1998-10-10,38272.500,45.0,7
2,22,103,1998-08-10,10080.000,45.0,7
3,22,104,1998-07-10,30240.000,45.0,7
4,64,101,1998-09-05,4536.000,16.0,7
5,64,102,1998-09-08,13608.000,16.0,7
6,31,102,1998-11-10,53946.000,55.5,8
7,31,103,1998-11-06,14208.000,55.5,8
8,31,104,1998-11-12,42624.000,55.5,8
9,74,103,1998-09-08,7344.000,25.5,9


In [18]:
# Step 1: Group by 'sid' in reserves_df and sum 'down_payment' values
total_down_payment = reserves_df.groupby("sid")["down_payment"].sum().reset_index()

# Step 2: Merge this total_down_payment with sailors_df on 'sid'
sailors_df = sailors_df.merge(total_down_payment, on="sid", how="left")

# Step 3: Update the 'money_spent' column with the total 'down_payment' values
sailors_df["money_spent"] = sailors_df["down_payment"]
sailors_df.drop(columns="down_payment", inplace=True)

# NaN means that the sailor has not made any reservations and hence has not spent any money
sailors_df["money_spent"] = sailors_df["money_spent"].fillna(0)

# Display the updated sailors_df
sailors_df

,sid,sname,rating,age,money_spent
0,22,dusting,7,45.0,91350.000
1,23,emilio,7,45.0,45675.000
2,24,scruntus,1,33.0,3168.000
3,29,brutus,1,33.0,0.000
4,31,lubber,8,55.5,110778.000
5,32,andy,8,25.5,0.000
6,35,figaro,8,55.5,64380.000
7,58,rusty,10,35.0,0.000
8,59,stum,8,25.5,34986.000
9,60,jit,10,35.0,77175.000


Boats4Pirates decides to automate the process of populating Sailor.money_spent when new reservation records are added to their database. 

In [21]:
reserves_df

,sid,bid,day,down_payment,age,rating
0,22,101,1998-10-10,12757.500,45.0,7
1,22,102,1998-10-10,38272.500,45.0,7
2,22,103,1998-08-10,10080.000,45.0,7
3,22,104,1998-07-10,30240.000,45.0,7
4,64,101,1998-09-05,4536.000,16.0,7
5,64,102,1998-09-08,13608.000,16.0,7
6,31,102,1998-11-10,53946.000,55.5,8
7,31,103,1998-11-06,14208.000,55.5,8
8,31,104,1998-11-12,42624.000,55.5,8
9,74,103,1998-09-08,7344.000,25.5,9


In [23]:
def add_reservation(sailors_df, boats_df, reserves_df, sid, bid, day):
    # Find market_value for the given bid in boats_df
    market_value = boats_df.loc[boats_df['bid'] == bid, 'market_value'].values[0]
    
    # Find rating and age for the given sid in sailors_df
    sailor_details = sailors_df.loc[sailors_df['sid'] == sid, ['rating', 'age']].iloc[0]
    rating, age = sailor_details['rating'], sailor_details['age']
    
    # Calculate down_payment
    down_payment = market_value * (rating / 10) * (age / 100)
    
    # Add new reservation to reserves_df
    new_reservation = {'sid': sid, 'bid': bid, 'day': day, 'down_payment': down_payment}
    reserves_df = reserves_df.append(new_reservation, ignore_index=True)
    
    # Update money_spent in sailors_df
    # First, recalculate total down_payment for the given sid in reserves_df
    total_down_payment = reserves_df[reserves_df['sid'] == sid]['down_payment'].sum()
    
    # Then, update money_spent for the given sid in sailors_df
    sailors_df.loc[sailors_df['sid'] == sid, 'money_spent'] = total_down_payment
    
    return sailors_df, boats_df, reserves_df


In [26]:
sailors_df, boats_df, reserves_df = add_reservation(sailors_df, boats_df, reserves_df, 22, 101, "2024-3-6")

reserves_df.head()


/var/folders/nx/n1nd7k552014npzf4p5zpxh00000gn/T/ipykernel_38925/2825881711.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reserves_df = reserves_df.append(new_reservation, ignore_index=True)


,sid,bid,day,down_payment,age,rating
0,22,101,1998-10-10,12757.5,45.0,7.0
1,22,102,1998-10-10,38272.5,45.0,7.0
2,22,103,1998-08-10,10080.0,45.0,7.0
3,22,104,1998-07-10,30240.0,45.0,7.0
4,64,101,1998-09-05,4536.0,16.0,7.0
